`conda env create -n suite2p_env -f optinist/conda/suite2p_env.yaml`

`!pip install imageio pynwb`

In [ ]:
import os
import sys
sys.path.append(os.path.abspath('..'))

from studio.wrappers.suite2p import file_convert, registration, roi
from studio.dataclass import ImageData

In [ ]:
sample_data = ImageData(['../sample_data/mouse2p_2_donotouse.tiff'], "")

In [ ]:
file_convert_params = {
    'nplanes': 1,
    'nchannels': 1,
    'force_sktiff': False,
    'batch_size': 500,
    'do_registration': 1
}

In [ ]:
ret = file_convert.suite2p_file_convert(sample_data, file_convert_params)
ret

In [ ]:
registration_params = {
    'frames_include': -1,
    'keep_movie_raw': False,
    'do_bidiphase': False,

    'smooth_sigma': 1.15,
    'smooth_sigma_time': 0,
    'bidiphase': 0,
    'maxregshift': 0.1,
    'maxregshiftNR': 5,
    'nonrigid': True,
    'block_size': [128, 128],
    'snr_thresh': 1.2,
    'functional_chan' : 1,
    'align_by_chan' : 1,
    'reg_tif': False,
    'th_badframes': 1.0,
    'diameter': 0,

    # 1P setting
    '1Preg': False,
    'spatial_hp_reg': 42,
    'pre_smooth': 0,
    'spatial_taper': 40,
    'bidi_corrected': False,
}

In [ ]:
ret = registration.suite2p_registration(ret['ops'], registration_params)

In [ ]:
suite2p_roi_params = {
    # main settings
    'tau':  1.0,              # this is the main parameter for deconvolution
    'fs': 10.0,             # sampling rate (PER PLANE e.g. for 12 plane recordings it will be around 2.5)

    # classification parameters
    'soma_crop': True,        # crop dendrites for cell classification stats like compactness

    # cell detection settings
    'high_pass': 100,         # running mean subtraction with window of size 'high_pass(use low values for 1P)
    'sparse_mode': True,      # whether or not to run sparse_mode
    'max_overlap': 0.75,      # cells with more overlap than this get removed during triage before refinement
    'nbinned': 5000,          # max number of binned frames for cell detection
    'spatial_scale': 0,       # 0: multi-scale; 1: 6 pixels 2: 12 pixels 3: 24 pixels 4: 48 pixels
    'threshold_scaling': 1.0, # adjust the automatically determined threshold by this scalar multiplier
    'max_iterations': 20,     # maximum number of iterations to do cell detection

    # 1P settings
    'spatial_hp_detect': 25,  # window for spatial high-pass filtering for neuropil subtraction before detection

    # output settings
    'preclassify': 0.,       # apply classifier before signal extraction with probability 0.3

    # ROI extraction parameters
    'allow_overlap': False,      # pixels that are overlapping are thrown out (False) or added to both ROIs (True)
    'inner_neuropil_radius': 2,  # number of pixels to keep between ROI and neuropil donut
    'min_neuropil_pixels': 350,  # minimum number of pixels in the neuropil

    # deconvolution settings
    'neucoeff': .7,          # neuropil coefficient
}

In [ ]:
ret = roi.suite2p_roi(ret['ops'], suite2p_roi_params)

In [ ]:
for roi in ret['nwbfile']['ROI']['roi_list']:
    print(roi.keys())